In [2]:
import numpy as np

'''N-그램 추출'''
import collections # 빈도수 계산
from nltk import ngrams # N-그램을 추출

# 분석할 파일을 지정
file_to_analyze = "./datasets/python-3.10.3-amd64.exe"

# 파일을 바이트로 읽기 위한 편의함수를 작성한다.
def read_file(file_path):
    # 이진 파일의 문자열을 읽는다.
    with open(file_path, "rb") as binary_file:
        data = binary_file.read()
    return data

# 바이트 문자열을 가져와 N-그램을 얻는 편의 함수를 정의한다.
def byte_sequence_to_Ngrams(byte_sequence, N):
    # 바이트 문자열에서 N-그램 리스트를 만든다.
    Ngrams = ngrams(byte_sequence, N)
    return list(Ngrams)

# 파일을 읽고 N-그램의 빈도수를 계산하는 함수를 작성한다.
def binary_file_to_Ngram_counts(file, N):
    # 이진 파일을 읽고, 이진 문자열에서 N-그램의 개수를 출력한다.
    filebyte_sequence = read_file(file)
    file_Ngrams = byte_sequence_to_Ngrams(filebyte_sequence, 4)
    return collections.Counter(file_Ngrams)

# 4-Gram을 추출한다.
extracted_Ngrams = binary_file_to_Ngram_counts(file_to_analyze, 4)

# 파일에서 가장 빈도수가 높은 4-그램 10개를 나열한다.
print(extracted_Ngrams.most_common(10))

[((0, 0, 0, 0), 24343), ((139, 240, 133, 246), 1920), ((32, 116, 111, 32), 1791), ((255, 255, 255, 255), 1668), ((108, 101, 100, 32), 1522), ((100, 32, 116, 111), 1519), ((97, 105, 108, 101), 1514), ((105, 108, 101, 100), 1513), ((70, 97, 105, 108), 1505), ((101, 100, 32, 116), 1503)]


In [3]:
'''최상의 N-그램 선택: 여러 개의 파일 중에서, 가장 많은 N-그램의 N을 찾는 작업'''
from os import listdir
from os.path import isfile, join

directories = ["Benign PE Samples", "Malicious PE Samples"]
N = 2

# 데이터셋의 모든 파일에서 모든 N-그램의 빈도수를 계산한다. -> 전역적으로 가장 빈도수가 높은 N-그램을 찾을 수 있다.
Ngram_counts_all_files = collections.Counter([])
for dataset_path in directories:
    all_samples = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]
    for sample in all_samples:
        file_path = join(dataset_path, sample)
        Ngram_counts_all_files += binary_file_to_Ngram_counts(file_path, N)

# 가장 빈도수가 높은 N-그램 K1=1000개를 리스트로 만든다.
K1 = 1000
K1_most_frequent_Ngrams = Ngram_counts_all_files.most_common(K1)
K1_most_frequent_Ngrams_list = [x[0] for x in K1_most_frequent_Ngrams]

# 샘플의 바이트 문자열에서 가장 흔한 N-그램의 출현한 횟수를 계산하는데 도움이 되는 메서드인 featurize_sample을 만든다.
def featurize_sample(sample, K1_most_frequent_Ngrams_list):
    # 샘플에서 특성 벡터를 만든다. 특성은 우리가 선택한 N-그램 K1개 빈도수다.
    K1 = len(K1_most_frequent_Ngrams_list)
    feature_vector = K1 * [0]
    file_Ngrams = binary_file_to_Ngram_counts(sample, N)
    for i in range(K1):
        feature_vector[i] = file_Ngrams[K1_most_frequent_Ngrams_list[i]]
    return feature_vector

# 디렉터리에 대해 반복하면서 샘플을 특성화하고자, 앞의 featurize_sample 함수를 사용한다. 또한 레이블의 집합도 만든다.
directories_with_labels = [("Benign PE Samples", 0), ("Malicious PE Samples", 1)]
X = []
Y = []
for dataset_path, label in directories_with_labels:
    all_samples = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]
    for sample in all_samples:
        file_path = join(dataset_path, sample)
        X.append(featurize_sample(file_path, K1_most_frequent_Ngrams_list))
        Y.append(label)

# 특성 선택에 사용할 라이브러리를 들여오고, 몇 개의 특성으로 줄일지 지정한다.
from sklearn.feature_selection import SelectKBest, mutual_info_classif, chi2
K2 = 10

# N-그램에 대해 다음 3가지 유형의 특성 선택을 수행한다.
# 빈도수: 가장 빈도수가 높은 N-그램을 선택한다.
X = np.asarray(X)
X_top_K2_freq = X[:,K2]

# 상호 정보: 상호 정보 알고리즘으로 가장 높은 순위에 오른 N-그램을 선택한다.
mi_selector = SelectKBest(mutual_info_classif, k=K2)
X_top_K2_mi = mi_selector.fit_transform(X, Y)

# 카이제곱: 카이제곱 알고리즘으로 가장 높은 순위에 오른 N-그램을 선택한다.
chi2_selector = SelectKBest(chi2, k=K2)
X_top_K2_chi2 = chi2_selector.fit_transform(X, Y)

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'Benign PE Samples'